# Train a Keras Model in Local mode
This notebook shows how to train and host a Keras Sequential model on SageMaker. The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

We'll compare trainig with file mode, pipe mode datasets and distributed training with Horovod

## Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
!pwd

In [ ]:
!pip install wget

In [ ]:
import wget

In [ ]:
!python generate_cifar10_tfrecords_v1.x.py --data-dir data/

## Create a training job using the sagemaker.TensorFlow estimator, running locally
To test that the code will work in SageMaker, we'll first use SageMaker local mode.

In [ ]:
from sagemaker.tensorflow import TensorFlow

import subprocess
instance_type = 'local'
if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
local_hyperparameters = {'epochs': 2, 'batch-size' : 64}

source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='cifar10_keras_main.py',
                       source_dir=source_dir,
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters=local_hyperparameters,
                       train_instance_count=1, 
                       train_instance_type=instance_type)

In [ ]:
local_inputs = {'train' : 'file://'+os.getcwd()+'/data/train', 
                'validation' : 'file://'+os.getcwd()+'/data/validation', 
                'eval' : 'file://'+os.getcwd()+'/data/eval'}
estimator.fit(local_inputs)